# Laboratorium 1


### Konfiguracja

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.collections as mcoll
import matplotlib.colors as mcolors
import json as js

### Interfejsy

[Dostępne kolory](https://matplotlib.org/3.1.1/gallery/color/named_colors.html)

[Dostępne znaczniki punktów](https://matplotlib.org/3.1.1/api/markers_api.html#module-matplotlib.markers)

In [2]:
class PointsCollection:
    def __init__(self, points = [], color = None, marker = None):
        self.points = np.array(points)
        self.color = color
        self.marker = marker
        
class LinesCollection:
    def __init__(self, lines = [], color = None):
        self.color = color
        self.lines = lines
        
    def add(self, line):
        self.lines.append(line)
        
    def get_collection(self):
        if self.color:
            return mcoll.LineCollection(self.lines, [mcolors.to_rgba(self.color)] * len(lines))
        else:
            return mcoll.LineCollection(self.lines)

class Plot:
    def __init__(self, points=[], lines=[], json = None):
        if json is None:
            self.points = points
            self.lines = lines
        else:
            self.points = [PointsCollection(pointsCol) for pointsCol in js.loads(json)["points"]]
            self.lines = [LinesCollection(linesCol) for linesCol in js.loads(json)["lines"]]
            
    def draw(self):
        ax = plt.axes()
        for collection in self.points:
            if collection.points.size > 0:
                ax.scatter(*zip(*collection.points), c=collection.color, marker=collection.marker)
        for collection in self.lines:
            ax.add_collection(collection.get_collection())
        ax.autoscale()
        plt.draw()
        
    def toJSON(self):
        return js.dumps({"points": [pointCol.points.tolist() for pointCol in self.points], 
                          "lines":[linesCol.lines for linesCol in self.lines]})

### Przykład użycia

In [3]:
%matplotlib notebook

plot = Plot([PointsCollection([(1, 2), (3, 1.5), (2, -1)]), 
             PointsCollection([(5, -2), (2, 2), (-2, -1)], color = 'green', marker = "^")], 
            [LinesCollection([[(-1,2),(-2,3)]])])
plot.draw()

<IPython.core.display.Javascript object>

### Rozwiązanie

In [4]:
%matplotlib notebook

from random import random, choice
from math import sin, cos, sqrt
from enum import Enum, auto
import math

signs = [-1, 1]

In [5]:
def generate_points_within_range(count, max):
    points = [(0.0, 0.0)] * count
    
    for i in range(count):    
        x = random() * max * choice(signs)
        y = random() * max * choice(signs)
        
        points[i] = (x, y)
        
    return points

def generate_points_a():
    return generate_points_within_range(10 ** 5, 10 ** 3)

def generate_points_b():
    return generate_points_within_range(10 ** 5, 10 ** 14)

In [6]:
def generate_points_c():
    radius = 100.0
    count = 1000
    points = [(0.0, 0.0)] * count
    
    for i in range(count):
        t = random()
        
        x = cos(2 * math.pi / t)
        y = sin(2 * math.pi / t)
        
        points[i] = (x * radius, y * radius)
    
    return points
    

In [7]:
def generate_points_d():
    count = 1000
    max = 1000
    
    point_a = (-1.0, 0.0)
    point_b = (1.0, 0.1)
    
    a = (point_a[0] - point_b[0]) / (point_a[1] - point_b[1])
    b = (-a) * point_a[0] + point_a[1]
    
    points = []    
    
    for _ in range(count):
        x = random() * max * choice(signs)
        points.append((x, a * x + b))
    
    return points    
    

In [ ]:
%matplotlib inline

points = generate_points_a()

plot = Plot([PointsCollection(points)])
plot.draw()

In [9]:
def det_1(a, b, c):
    return a[0] * (b[1] - c[1]) - a[1] * (b[0] - c[0]) + (b[0] * c[1] - c[0] * b[1])
    
def det_2(a, b, c):
    return (a[0] - c[0]) * (b[1] - c[1]) - (a[1] - c[1]) * (b[0] - c[0])

def det2x2(a, b):
    return (a[0]*b[1]) - (a[1]*b[0])

In [10]:
class Orientation(Enum):
    LEFT = auto()
    RIGHT = auto()
    COLLINEAR = auto()

In [11]:
def classify_point(c, epsilon, det):
    a = (-1.0, 0.0)
    b = (1.0, 0.1)
    
    determinant = det(a, b, c)
    orientation = Orientation.COLLINEAR
    
    if determinant > epsilon:
        orientation = Orientation.LEFT
    elif determinant < -epsilon:
        orientation = Orientation.RIGHT
    
    return orientation

In [12]:
def classify(points, epsilon, det):
    
    left, collinear, right = [], [], []
    
    for point in points:
        orientation = classify_point(point, epsilon, det)
        
        if orientation == Orientation.LEFT:
            left.append(point)
        elif orientation == Orientation.RIGHT:
            right.append(point)
        else:
            collinear.append(point)

    return (left, collinear, right)

In [13]:
epsilon_0 = 1
epsilon_1 = 10 ** (-3)
epsilon_2 = 10 ** (-6)
epsilon_3 = 10 ** (-9)

In [14]:
%matplotlib notebook

def show_results(points, epsilon, det):
    (left, collinear, right) = classify(points, epsilon, det)

    plot = Plot([
        PointsCollection(left),
        PointsCollection(right, color='red'),
        PointsCollection(collinear, color='green')])
    
    print("Classified LEFT: {0} \nClassified RIGHT: {1} \nClassified COLLINEAR: {2}"
          .format(len(left), len(right), len(collinear)))

    plot.draw()

In [15]:
### Classification of set A
points_a = generate_points_a()

In [16]:
%matplotlib notebook

show_results(points_a, epsilon_0, det_1)

Classified LEFT: 50134 
Classified RIGHT: 49818 
Classified COLLINEAR: 48


<IPython.core.display.Javascript object>

In [17]:
%matplotlib notebook

show_results(points_a, epsilon_1, det_1)

Classified LEFT: 50161 
Classified RIGHT: 49839 
Classified COLLINEAR: 0


<IPython.core.display.Javascript object>

In [18]:
%matplotlib notebook

show_results(points_a, epsilon_1, det_2)

Classified LEFT: 50161 
Classified RIGHT: 49839 
Classified COLLINEAR: 0


<IPython.core.display.Javascript object>

In [19]:
%matplotlib notebook

show_results(points_a, epsilon_2, det_1)

Classified LEFT: 50161 
Classified RIGHT: 49839 
Classified COLLINEAR: 0


<IPython.core.display.Javascript object>

In [20]:
%matplotlib notebook

show_results(points_a, epsilon_2, det_2)

Classified LEFT: 50161 
Classified RIGHT: 49839 
Classified COLLINEAR: 0


<IPython.core.display.Javascript object>

In [21]:
%matplotlib notebook

show_results(points_a, epsilon_3, det_1)

Classified LEFT: 50161 
Classified RIGHT: 49839 
Classified COLLINEAR: 0


<IPython.core.display.Javascript object>

In [22]:
%matplotlib notebook

show_results(points_a, epsilon_3, det_2)

Classified LEFT: 50161 
Classified RIGHT: 49839 
Classified COLLINEAR: 0


<IPython.core.display.Javascript object>

In [23]:
### Classification of set B

points_b = generate_points_b()

In [24]:
%matplotlib notebook

show_results(points_b, epsilon_0, det_1)

Classified LEFT: 49971 
Classified RIGHT: 50029 
Classified COLLINEAR: 0


<IPython.core.display.Javascript object>

In [25]:
%matplotlib notebook

show_results(points_b, epsilon_1, det_1)

Classified LEFT: 49971 
Classified RIGHT: 50029 
Classified COLLINEAR: 0


<IPython.core.display.Javascript object>

In [26]:
%matplotlib notebook

show_results(points_b, epsilon_1, det_2)

Classified LEFT: 49969 
Classified RIGHT: 50021 
Classified COLLINEAR: 10


<IPython.core.display.Javascript object>

In [27]:
%matplotlib notebook

show_results(points_b, epsilon_2, det_1)

Classified LEFT: 49971 
Classified RIGHT: 50029 
Classified COLLINEAR: 0


<IPython.core.display.Javascript object>

In [28]:
%matplotlib notebook

show_results(points_b, epsilon_2, det_2)

Classified LEFT: 49969 
Classified RIGHT: 50021 
Classified COLLINEAR: 10


<IPython.core.display.Javascript object>

In [29]:
%matplotlib notebook

show_results(points_b, epsilon_3, det_1)

Classified LEFT: 49971 
Classified RIGHT: 50029 
Classified COLLINEAR: 0


<IPython.core.display.Javascript object>

In [30]:
%matplotlib notebook

show_results(points_b, epsilon_3, det_2)

Classified LEFT: 49969 
Classified RIGHT: 50021 
Classified COLLINEAR: 10


<IPython.core.display.Javascript object>

In [31]:
### Classification of set C

points_c = generate_points_c()

In [32]:
%matplotlib notebook

show_results(points_c, epsilon_0, det_1)

Classified LEFT: 618 
Classified RIGHT: 376 
Classified COLLINEAR: 6


<IPython.core.display.Javascript object>

In [33]:
%matplotlib notebook

show_results(points_c, epsilon_0, det_1)

Classified LEFT: 618 
Classified RIGHT: 376 
Classified COLLINEAR: 6


<IPython.core.display.Javascript object>

In [34]:
%matplotlib notebook

show_results(points_c, epsilon_1, det_1)

Classified LEFT: 621 
Classified RIGHT: 379 
Classified COLLINEAR: 0


<IPython.core.display.Javascript object>

In [35]:
%matplotlib notebook

show_results(points_c, epsilon_1, det_2)

Classified LEFT: 621 
Classified RIGHT: 379 
Classified COLLINEAR: 0


<IPython.core.display.Javascript object>

In [36]:
%matplotlib notebook

show_results(points_c, epsilon_2, det_1)

Classified LEFT: 621 
Classified RIGHT: 379 
Classified COLLINEAR: 0


<IPython.core.display.Javascript object>

In [37]:
%matplotlib notebook

show_results(points_c, epsilon_2, det_2)

Classified LEFT: 621 
Classified RIGHT: 379 
Classified COLLINEAR: 0


<IPython.core.display.Javascript object>

In [38]:
%matplotlib notebook

show_results(points_c, epsilon_3, det_1)

Classified LEFT: 621 
Classified RIGHT: 379 
Classified COLLINEAR: 0


<IPython.core.display.Javascript object>

In [39]:
%matplotlib notebook

show_results(points_c, epsilon_3, det_2)

Classified LEFT: 621 
Classified RIGHT: 379 
Classified COLLINEAR: 0


<IPython.core.display.Javascript object>

In [40]:
### Classification os set D

points_d = generate_points_d()

In [41]:
%matplotlib notebook

show_results(points_d, epsilon_0, det_1)

Classified LEFT: 466 
Classified RIGHT: 534 
Classified COLLINEAR: 0


<IPython.core.display.Javascript object>

In [42]:
%matplotlib notebook

show_results(points_d, epsilon_1, det_1)

Classified LEFT: 466 
Classified RIGHT: 534 
Classified COLLINEAR: 0


<IPython.core.display.Javascript object>

In [43]:
%matplotlib notebook

show_results(points_d, epsilon_1, det_2)

Classified LEFT: 466 
Classified RIGHT: 534 
Classified COLLINEAR: 0


<IPython.core.display.Javascript object>

In [44]:
%matplotlib notebook

show_results(points_d, epsilon_2, det_1)

Classified LEFT: 466 
Classified RIGHT: 534 
Classified COLLINEAR: 0


<IPython.core.display.Javascript object>

In [45]:
%matplotlib notebook

show_results(points_d, epsilon_2, det_2)

Classified LEFT: 466 
Classified RIGHT: 534 
Classified COLLINEAR: 0


<IPython.core.display.Javascript object>

In [46]:
%matplotlib notebook

show_results(points_d, epsilon_3, det_1)

Classified LEFT: 466 
Classified RIGHT: 534 
Classified COLLINEAR: 0


<IPython.core.display.Javascript object>

In [47]:
%matplotlib notebook

show_results(points_d, epsilon_3, det_2)

Classified LEFT: 466 
Classified RIGHT: 534 
Classified COLLINEAR: 0


<IPython.core.display.Javascript object>